In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import torch
import torch.nn as nn
import torchaudio as ta
import tensorflow as tf
from tensorflow.keras import layers
import random
import os
import time


In [ ]:
data = []
for dirname, _, filenames in os.walk('/kaggle/input/transform-speech-codes-into-speech-signal-dim0446/audiocodes2speech_dim0494/train'):
     for filename in filenames:
        name = os.path.join(dirname, filename)
        oi = name[0:len(name)-3]
        ext = name[len(name)-3:len(name)]
        if ext == "pth":
            data.append(oi)        
            #print(name)
        #print(oi)

In [ ]:

#df.head()

train_ds = tf.data.Dataset.from_tensor_slices(data)




def pre_process(s):
    #print("antes")
    #print(s.numpy().decode())    
    #print("depois")
#     file = torch.load(s.numpy().decode()+"pth", torch.device('cpu'))
    file = torch.load(s.numpy().decode()+"pth")
    waveform, sample_rate = librosa.load(s.numpy().decode()+"wav", sr=None)
    
    nr = random.randrange(file.shape[1] - 32)
    
    file = file[0][nr:nr+32]
    #print(waveform.size)
    #print(nr*1024)
    waveform = waveform[nr*1024:(nr+32)*1024]
    return (file, waveform)

#for elemen in train_ds:
    #file, waveform = pre_process(elemen)
    #print(file.numpy().size)
    #print(waveform.size)
    
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# This method returns a helper function to compute cross entropy loss
tf.keras.losses.BinaryCrossentropy(from_logits=True)
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
    
# Embaralhamento e criação de lotes com preenchimento
#train_ds = train_ds.shuffle(23).padded_batch(16)
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [ ]:
#gerador

def make_generator_model():
    model = tf.keras.Sequential()
    
    # Camada de embedding
    model.add(layers.Embedding(8192,1024))

    # Conv21DTranspose - 1
    model.add(layers.Conv1DTranspose(512, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 2
    model.add(layers.Conv1DTranspose(256, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 3
    model.add(layers.Conv1DTranspose(128, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 4
    model.add(layers.Conv1DTranspose(64, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 5
    model.add(layers.Conv1DTranspose(32, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 6
    model.add(layers.Conv1DTranspose(16, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 7
    model.add(layers.Conv1DTranspose(8, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 8
    model.add(layers.Conv1DTranspose(4, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 9
    model.add(layers.Conv1DTranspose(2, 4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Conv21DTranspose - 10
    model.add(layers.Conv1DTranspose(1, 4, strides=2, padding="same", activation = tf.keras.activations.tanh))
    model.add(layers.BatchNormalization())
    

    return model

# Gere um áudio com o gerador adaptado
#print(datapth[0].shape)
#v=np.array([datapth[0][0][0:32]],ndmin=2)
#print(v.shape)
generator = make_generator_model()
#for elemen in train_ds:
    #file, waveform = pre_process(elemen)
    #v=np.array(file,ndmin=2)
    #print(v[0])
    #generated_audio = generator(v, training=False)
    #print(waveform.size)
#generated_audio = audio_generator(v, training=False)

#print(generated_audio.shape)
#print(generated_audio)



In [ ]:
#discriminador
def make_discriminator_model():    
    model = tf.keras.Sequential()
    #model.add(layers.Input(shape=(336896, 1)))
    model.add(layers.Conv1D(64, 3, strides=1, padding="same",
                                     input_shape=[32768, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))    
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Conv1D(128, 3, strides=1, padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Conv1D(256, 3, strides=1, padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.MaxPooling1D(2))
    
    model.add(layers.Conv1D(512, 3, strides=1, padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.Dense(128))
    model.add(layers.Dense(1))
    model.add(layers.Softmax())
    
    return model

# modelo discriminador
discriminator = make_discriminator_model()
#generated_number = audio_discriminator(generated_audio, training=False)
#print(generated_number)

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(file,waveform):
    noise = tf.random.normal([256, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:        
        generated_images = generator(file, training=True)
        
        #print(generated_images)
        oi = tf.reshape(waveform, (1, 32768, 1))
        #print(oi)

        real_output = discriminator(oi, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    print("ate o fim")
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for elemen in dataset:
        file, waveform = pre_process(elemen)
#         print(file[0])
        v=np.array(file.to('cpu'),ndmin=2)
        train_step(v,waveform)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)
train(train_ds, 50)
